In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
%%time
train_metadata_kaggle = dd.read_csv('mydata_train_metadata.csv')
test_metadata_kaggle = dd.read_csv('mydata_test_metadata.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 139) (3492890, 138)
CPU times: user 3min 21s, sys: 11.9 s, total: 3min 33s
Wall time: 38.8 s


In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 34s, sys: 12.3 s, total: 3min 46s
Wall time: 40.1 s


In [6]:
%%time
test_metadata_kaggle = test_metadata_kaggle.reset_index(drop=True)
test_metadata = test_metadata.reset_index(drop=True)

CPU times: user 6.5 s, sys: 9.07 s, total: 15.6 s
Wall time: 15.6 s


In [7]:
print(train_metadata_kaggle['object_id'].equals(train_metadata['object_id']))

True


In [8]:
print(test_metadata_kaggle['object_id'].equals(test_metadata['object_id']))

True


In [9]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [10]:
temp_columns = ['object_id','A0_max_flux','A0_min_flux','A0_mean_flux','A0_median_flux','A0_std_flux',
'A1_max_flux','A1_min_flux','A1_mean_flux','A1_median_flux','A1_std_flux',
'A2_max_flux','A2_min_flux','A2_mean_flux','A2_median_flux','A2_std_flux',
'A3_max_flux','A3_min_flux','A3_mean_flux','A3_median_flux','A3_std_flux',
'A4_max_flux','A4_min_flux','A4_mean_flux','A4_median_flux','A4_std_flux',
'A5_max_flux','A5_min_flux','A5_mean_flux','A5_median_flux','A5_std_flux']

In [11]:
train_metadata.head()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_pl

In [12]:
train_metadata.drop([x for x in train_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()
test_metadata.drop([x for x in test_metadata.columns if x not in temp_columns], axis=1,inplace=True)
gc.collect()

7

In [62]:
used_columns1 = ['A5_median_mean_avg_flux',
 'A4_mean_min_avg_flux',
 'A0_max_mean_avg_flux']

In [13]:
print(train_metadata.shape,test_metadata.shape)

(7848, 31) (3492890, 31)


In [14]:
######################################TRAIN####################################################3
#A0
train_metadata['A0_max_min_avg_flux'] = (train_metadata['A0_max_flux'] + train_metadata['A0_min_flux']) / 2
train_metadata['A0_max_mean_avg_flux'] = (train_metadata['A0_max_flux'] + train_metadata['A0_mean_flux'])/ 2
train_metadata['A0_max_median_avg_flux'] = (train_metadata['A0_max_flux'] + train_metadata['A0_median_flux'])/ 2
train_metadata['A0_median_mean_avg_flux'] = (train_metadata['A0_median_flux'] + train_metadata['A0_mean_flux'])/ 2
train_metadata['A0_median_min_avg_flux'] = (train_metadata['A0_median_flux'] + train_metadata['A0_min_flux'])/ 2
train_metadata['A0_mean_min_avg_flux'] = (train_metadata['A0_mean_flux'] + train_metadata['A0_min_flux'])/ 2




In [15]:
#A1
train_metadata['A1_max_min_avg_flux'] = (train_metadata['A1_max_flux'] + train_metadata['A1_min_flux']) / 2
train_metadata['A1_max_mean_avg_flux'] = (train_metadata['A1_max_flux'] + train_metadata['A1_mean_flux'])/ 2
train_metadata['A1_max_median_avg_flux'] = (train_metadata['A1_max_flux'] + train_metadata['A1_median_flux'])/ 2
train_metadata['A1_median_mean_avg_flux'] = (train_metadata['A1_median_flux'] + train_metadata['A1_mean_flux'])/ 2
train_metadata['A1_median_min_avg_flux'] = (train_metadata['A1_median_flux'] + train_metadata['A1_min_flux'])/ 2
train_metadata['A1_mean_min_avg_flux'] = (train_metadata['A1_mean_flux'] + train_metadata['A1_min_flux'])/ 2

#A2
train_metadata['A2_max_min_avg_flux'] = (train_metadata['A2_max_flux'] + train_metadata['A2_min_flux']) / 2
train_metadata['A2_max_mean_avg_flux'] = (train_metadata['A2_max_flux'] + train_metadata['A2_mean_flux'])/ 2
train_metadata['A2_max_median_avg_flux'] = (train_metadata['A2_max_flux'] + train_metadata['A2_median_flux'])/ 2
train_metadata['A2_median_mean_avg_flux'] = (train_metadata['A2_median_flux'] + train_metadata['A2_mean_flux'])/ 2
train_metadata['A2_median_min_avg_flux'] = (train_metadata['A2_median_flux'] + train_metadata['A2_min_flux'])/ 2
train_metadata['A2_mean_min_avg_flux'] = (train_metadata['A2_mean_flux'] + train_metadata['A2_min_flux'])/ 2

#A3
train_metadata['A3_max_min_avg_flux'] = (train_metadata['A3_max_flux'] + train_metadata['A3_min_flux']) / 2
train_metadata['A3_max_mean_avg_flux'] = (train_metadata['A3_max_flux'] + train_metadata['A3_mean_flux'])/ 2
train_metadata['A3_max_median_avg_flux'] = (train_metadata['A3_max_flux'] + train_metadata['A3_median_flux'])/ 2
train_metadata['A3_median_mean_avg_flux'] = (train_metadata['A3_median_flux'] + train_metadata['A3_mean_flux'])/ 2
train_metadata['A3_median_min_avg_flux'] = (train_metadata['A3_median_flux'] + train_metadata['A3_min_flux'])/ 2
train_metadata['A3_mean_min_avg_flux'] = (train_metadata['A3_mean_flux'] + train_metadata['A3_min_flux'])/ 2

#A4
train_metadata['A4_max_min_avg_flux'] = (train_metadata['A4_max_flux'] + train_metadata['A4_min_flux']) / 2
train_metadata['A4_max_mean_avg_flux'] = (train_metadata['A4_max_flux'] + train_metadata['A4_mean_flux'])/ 2
train_metadata['A4_max_median_avg_flux'] = (train_metadata['A4_max_flux'] + train_metadata['A4_median_flux'])/ 2
train_metadata['A4_median_mean_avg_flux'] = (train_metadata['A4_median_flux'] + train_metadata['A4_mean_flux'])/ 2
train_metadata['A4_median_min_avg_flux'] = (train_metadata['A4_median_flux'] + train_metadata['A4_min_flux'])/ 2
train_metadata['A4_mean_min_avg_flux'] = (train_metadata['A4_mean_flux'] + train_metadata['A4_min_flux'])/ 2

#A5
train_metadata['A5_max_min_avg_flux'] = (train_metadata['A5_max_flux'] + train_metadata['A5_min_flux']) / 2
train_metadata['A5_max_mean_avg_flux'] = (train_metadata['A5_max_flux'] + train_metadata['A5_mean_flux'])/ 2
train_metadata['A5_max_median_avg_flux'] = (train_metadata['A5_max_flux'] + train_metadata['A5_median_flux'])/ 2
train_metadata['A5_median_mean_avg_flux'] = (train_metadata['A5_median_flux'] + train_metadata['A5_mean_flux'])/ 2
train_metadata['A5_median_min_avg_flux'] = (train_metadata['A5_median_flux'] + train_metadata['A5_min_flux'])/ 2
train_metadata['A5_mean_min_avg_flux'] = (train_metadata['A5_mean_flux'] + train_metadata['A5_min_flux'])/ 2

In [60]:
test_metadata['A5_median_mean_avg_flux'] = (test_metadata['A5_median_flux'] + test_metadata['A5_mean_flux'])/ 2
test_metadata['A4_mean_min_avg_flux'] = (test_metadata['A4_mean_flux'] + test_metadata['A4_min_flux'])/ 2
test_metadata['A0_max_mean_avg_flux'] = (test_metadata['A0_max_flux'] + test_metadata['A0_mean_flux'])/ 2

In [16]:
print(train_metadata.shape)

(7848, 67)


In [18]:
print(train_metadata_kaggle.shape)

(7848, 139)


In [19]:
test_id = test_metadata_kaggle['object_id']

In [20]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [21]:
train_metadata[list(train_metadata.columns)[-36:]].head()

,A0_max_min_avg_flux,A0_max_mean_avg_flux,A0_max_median_avg_flux,A0_median_mean_avg_flux,A0_median_min_avg_flux,A0_mean_min_avg_flux,A1_max_min_avg_flux,A1_max_mean_avg_flux,A1_max_median_avg_flux,A1_median_mean_avg_flux,A1_median_min_avg_flux,A1_mean_min_avg_flux,A2_max_min_avg_flux,A2_max_mean_avg_flux,A2_max_median_avg_flux,A2_median_mean_avg_flux,A2_median_min_avg_flux,A2_mean_min_avg_flux,A3_max_min_avg_flux,A3_max_mean_avg_flux,A3_max_median_avg_flux,A3_median_mean_avg_flux,A3_median_min_avg_flux,A3_mean_min_avg_flux,A4_max_min_avg_flux,A4_max_mean_avg_flux,A4_max_median_avg_flux,A4_median_mean_avg_flux,A4_median_min_avg_flux,A4_mean_min_avg_flux,A5_max_min_avg_flux,A5_max_mean_avg_flux,A5_max_median_avg_flux,A5_median_mean_avg_flux,A5_median_min_avg_flux,A5_mean_min_avg_flux
0,4.134795,60.964128,57.583793,-6.634890,-63.464222,-60.083887,-219.686830,137.413170,86.313170,-436.90000,-794.00000,-742.90000,-35.007720,238.919000,173.092280,-199.973280,-473.90000,-408.073280,-42.381470,162.316780,141.818530,-141.601750,-346.30000,-325.801750,-20.115380,162.999573,139.206185,-79.747982,-262.862935,-239.069547,-22.313480,165.369145,146.331915,-66.487080,-254.169705,-235.132475
1,-0.112674,5.894716,5.706512,-2.908601,-8.915991,-8.727788,-1.295490,4.054511,4.283760,-0.79075,-6.14075,-6.37000,0.229521,4.867402,5.205521,-0.456119,-5.09400,-5.432119,-0.534842,5.171675,5.628208,-0.530433,-6.23695,-6.693483,-1.229433,4.463836,4.517879,-0.846219,-6.539488,-6.593531,0.279861,6.488356,6.153937,-2.128593,-8.337088,-8.002669
2,1.241103,2.947043,2.983129,-0.011994,-1.717933,-1.754020,1.150054,2.917104,2.932254,0.15625,-1.61080,-1.62595,9.072355,11.697790,10.743205,1.446285,-1.17915,-0.224565,14.067050,18.404132,17.116350,1.948382,-2.38870,-1.100918,17.661835,22.734354,21.082167,2.656541,-2.415978,-0.763791,14.075125,25.924728,24.926354,3.541022,-8.308582,-7.310207
3,7.069840,9.905776,9.535372,1.427118,-1.408817,-1.038413,94.312150,98.982150,96.566150,3.30400,-1.36600,1.05000,109.317605,115.253371,110.609755,5.067916,-0.86785,3.775766,99.152850,108.831812,102.305850,7.886963,-1.79200,4.733962,84.069328,98.383778,92.451635,7.202293,-7.112156,-1.180014,65.631951,76.129714,72.131422,6.747846,-3.749916,0.248376
4,-0.736888,2.995937,2.955977,0.620987,-3.111838,-3.071878,17.273589,20.901589,19.162089,2.89350,-0.73450,1.00500,52.293845,58.457829,53.780345,5.566484,-0.59750,4.079984,68.509200,75.452478,70.416200,6.050277,-0.89300,4.143277,63.528219,76.753146,72.672601,5.825557,-7.399370,-3.318825,49.148523,58.027163,55.565429,4.435007,-4.443634,-1.981899


In [ ]:
#[0.5764, 0.54968, 0.59261, 0.56469, 0.5659]
"""
0 0.5407134604014688
1 0.5505294469507865
2 0.5930505215690117
3 0.5412398994685668
4 0.5650806630208185
MULTI WEIGHTED LOG LOSS : 0.55802
"""

In [59]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')

print(temp.shape)
y = temp['target']
del temp['target']
classes = sorted(y.unique())

class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'n_estimators': 4000, 
'subsample_freq': 6, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 40,
'min_child_weight': 150, 
'min_split_gain': 0.15, 
'num_leaves': 6, 
'reg_alpha': 0.1, 
'reg_lambda': 0.001, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 142)
0 0.5742077279623574
1 0.5526378937737927
2 0.5895248906291187
3 0.5582664456933794
4 0.564887714048932
MULTI WEIGHTED LOG LOSS : 0.56791 
CPU times: user 12min 21s, sys: 2.19 s, total: 12min 23s
Wall time: 1min 41s


In [ ]:
#final_dict2[final_dict2['column_name'].isin(most_imp_ones)]
#final_dict2[final_dict2['fold_sum'] == 1]

In [ ]:
#final_dict2.head()

In [ ]:
#final_dict2.to_csv('final_features3.csv',index=False)

In [61]:
#modify to work with kfold
#def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
def smoteAdataset(Xig_train, yig_train, Xig_test, yig_test):
    
        
    sm=SMOTE(random_state=51)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

        
    return Xig_train_res, pd.Series(yig_train_res), Xig_test, pd.Series(yig_test)

In [64]:
%%time
final_dict = {}

loss_list = []
temp = train_metadata_kaggle.copy()

#temp = temp.merge(train_metadata[['object_id'] + most_imp_ones2 ],on = 'object_id',how = 'left')

temp = temp.merge(train_metadata[['object_id'] + used_columns1  ],on = 'object_id',how = 'left')


print(temp.shape)
temp.fillna(0, inplace=True)

y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'n_estimators': 4000, 
'subsample_freq': 6, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 40,
'min_child_weight': 150, 
'min_split_gain': 0.15, 
'num_leaves': 6, 
'reg_alpha': 0.1, 
'reg_lambda': 0.001, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    trn_xa, trn_y, val_xa, val_y=smoteAdataset(trn_x.values, trn_y.values, val_x.values, val_y.values)
    trn_x=pd.DataFrame(data=trn_xa, columns=trn_x.columns)
    val_x=pd.DataFrame(data=val_xa, columns=val_x.columns)
    
    print(trn_x.shape,trn_y.shape,val_x.shape,val_y.shape)
    
    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

(7848, 142)
(25900, 140) (25900,) (1574, 140) (1574,)
0 0.5348824750716419
(25900, 140) (25900,) (1572, 140) (1572,)
1 0.5459372861798201
(25900, 140) (25900,) (1571, 140) (1571,)
2 0.5832585551293566
(25914, 140) (25914,) (1567, 140) (1567,)
3 0.5399954106964144
(25914, 140) (25914,) (1564, 140) (1564,)
4 0.5627263786677553
MULTI WEIGHTED LOG LOSS : 0.55324 
CPU times: user 30min 54s, sys: 2.76 s, total: 30min 56s
Wall time: 4min 29s


In [ ]:
#used_columns = used_columns1 + used_columns2 + used_columns3 

In [65]:
print(train_metadata.shape,test_metadata.shape)

(7848, 67) (3492890, 34)


In [66]:
#train_metadata.drop([x for x in train_metadata.columns if x not in ['object_id'] + used_columns ] ,axis = 1,inplace=True)

In [67]:
#test_metadata = test_metadata[[x for x in test_metadata.columns if x in ['object_id'] + used_columns ]]

In [68]:
train_metadata_kaggle = train_metadata_kaggle.merge(train_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')
test_metadata_kaggle = test_metadata_kaggle.merge(test_metadata[['object_id'] + used_columns1 ],on = 'object_id',how = 'left')

In [69]:
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 142) (3492890, 141)


In [71]:
test_metadata_kaggle.tail()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux,div_A4_A2_median_min_flux,div_A5_A2_median_min_flux,div_A5_A2_minus_1_sigma,div_A5_A4_median_mean_flux,div_A3_A0_plus_1_sigma,div_A4_A1_minus_1_sigma,A5_median_mean_avg_flux,A4_mean_min_avg_flux,A0_max_mean_avg_flux
3492885,130787966,-49.490440,75.235832,3.905430,1.703813,21.009541,0.625654,1.341227,70.066055,14.846084,10.951475,12.628468,1.307980,0.013793,374.420803,8.678639,5.954086e+03,7.560569,15.902124,124.726272,31.936629,7.843372,35.655183,32.200453,0.154555,-0.174445,23.565507,13.595040,6.803259,2.380728,63.739853,39.279845,3.023876,1.647382,82.545904,81.181195,0.670890,0.882368,187.846479,74.176357,3.792912,1.576256,172.934431,184.079710,-0.297825,0.291885,145.0,8.0,10.0,20.061909,0.151706,4.0,43.0,89.761400,89.761400,89.7614,0.4493,0.9954,41.9836,0.036,1.937777,2.277911,1.215719,35.655180,2.377012,0.0000,0.0909,0.0,0.0,0.000,0.00000,0.5557,-2.620740,0.0,5.474970e-13,0.103244,2.16581,0.050748,0.683935,2.26490,0.829141,2.439200,2.16581,0.500000,0.953504,0.808853,0.966782,2.32229,0.185745,19.47180,0.427032,10.79380,0.000000,1.000000,1.000000,0,0,0,0,-20.260

In [72]:
test_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,haversine,latlon1,hostgal_photoz_certain,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected,__flux_percentile_ratio_mid50___5_,__flux_percentile_ratio_mid65___2_,__median_absolute_deviation___2_,__qso_log_chi2_qsonu___0_,__stetson_k___1_,__freq1_signif___2_,__stetson_k___2_,__freq3_amplitude1___1_,__median_absolute_deviation___2_.1,__percent_close_to_median___2_,__freq_varrat___5_,__freq_varrat___4_,__qso_log_chi2_qsonu___3_,__qso_log_chi2_qsonu___1_,__qso_log_chi2_qsonu___5_,__std___4_,__freq_varrat___3_,__amplitude___2_,outlierScore,hipd,lipd,highEnergy_transitory_1.0_TF,highEnergy_transitory_1.5_TF,lowEnergy_transitory_1.0_TF,lowEnergy_transitory_1.5_TF,A1_minus_3_sigma,A5_max_median_diff_flux,A5_minus_3_sigma,A5_max_mean_diff_flux,diff_A5_A4_max_min_flux,diff_A2_A1_max_min_flux,diff_A3_A2_median_min_flux,diff_A5_A4_max_median_flux,diff_A4_A3_max_median_flux,diff_A2_A0_median_min_flux,diff_A4_A3_max_mean_flux,diff_A5_A2_max_mean_flux,diff_A5_A3_max_mean_flux,diff_A4_A0_median_mean_flux,diff_A5_A4_max_mean_flux,diff_A2_A1_max_median_flux,diff_A5_A2_max_median_flux,diff_A5_A4_median_min_flux,diff_A4_A0_median_min_flux,diff_A4_A1_max_median_flux,diff_A4_A2_max_median_flux,diff_A5_A4_minus_1_sigma,diff_A5_A3_median_min_flux,diff_A5_A3_max_median_flux,diff_A3_A1_minus_1_sigma,diff_A3_A0_median_min_flux,diff_A3_A0_plus_1_sigma,diff_A1_A0_median_min_flux,diff_A4_A2_mean_min_flux,diff_A5_A1_plus_1_sigma,diff_A4_A1_median_mean_flux,diff_A3_A2_max_median_flux,diff_A5_A1_median_mean_flux,div_A4_A2_median_min_flux,div_A5_A2_median_min_flux,div_A5_A2_minus_1_sigma,div_A5_A4_median_mean_flux,div_A3_A0_plus_1_sigma,div_A4_A1_minus_1_sigma,A5_median_mean_avg_flux,A4_mean_min_avg_flux,A0_max_mean_avg_flux
0,13,-12.680235,42.765503,3.997127,0.616561,9.149645,2.037355,0.691634,11.257108,2.461810,1.972972,1.718101,1.826388,0.157576,7806.412424,4.771625,1.896346e+05,5.396523,24.292155,55.445738,13.871398,2.282455,29.002872,37.684425,-0.247160,0.147622,92.039690,79.990817,10.567412,3.117684,164.640622,139.733762,4.512783,2.171455,300.547278,246.788411,2.951479,1.806797,408.305525,349.008202,0.094963,1.153076,324.515880,251.116361,0.072799,0.804929,330.0,15.0,32.0,4.426774,0.017885,9.0,222.0,2.350061,2.350061,119.8531,0.3193,0.0542,41.1123,0.019,1.851382,3.049709,0.337084,29.002872,0.402818,0.0000,0.07690,11.0,0.3137,0.3254,0.11770,0.7114,-1.778855,6.0,0.005627,0.528087,1.384690,-0.007597,0.610190,4.81984,0.681284,1.783390,1.384690,0.711538,0.255030,0.090550,3.499440,2.788160,1.510790,13.24940,0.124063,13.15420,1.0,1.0,1.0,1,1,1,0,-13.032169,33.023636,-29.949

In [73]:
%%time
train_metadata_kaggle.to_csv('mydata_train_metadata.csv',index=False)
test_metadata_kaggle.to_csv('mydata_test_metadata.csv',index=False)

CPU times: user 9min 58s, sys: 6.83 s, total: 10min 5s
Wall time: 10min 6s


In [74]:
imp_df.sort_values('gain',ascending=False).head()

,feature,gain,fold
57,distmod,1538,5
55,hostgal_photoz,1477,5
54,mjd_diff_det,1385,5
61,hostgal_photoz_certain,829,5
56,hostgal_photoz_err,751,5


In [75]:
temp_test = test_metadata_kaggle.copy()

In [76]:
del temp_test['object_id']

In [77]:
temp_test.fillna(0,inplace = True)

In [78]:
print(temp.shape,temp_test.shape)

(7848, 140) (3492890, 140)


In [79]:
list(temp.columns) == list(temp_test.columns)

True

In [80]:
%%time
test_pred0 = pd.DataFrame()
test_pred1 = pd.DataFrame()
test_pred2 = pd.DataFrame()
test_pred3 = pd.DataFrame()
test_pred4 = pd.DataFrame()

list_of_df = [test_pred0,test_pred1,test_pred2,test_pred3,test_pred4]

for num,c in enumerate(clfs):
    print(num)
    for k in range(0,len(temp_test),500000):
        test_pred = pd.DataFrame(c.predict_proba(temp_test[ k:k+500000] ))
        list_of_df[num] = pd.concat([list_of_df[num],test_pred],axis=0)
        del test_pred

0
1
2
3
4
CPU times: user 3h 13min 56s, sys: 33.2 s, total: 3h 14min 30s
Wall time: 25min 31s


In [81]:
test_pred2 = pd.DataFrame()
test_pred2 = (list_of_df[0] + list_of_df[1] + list_of_df[2] + list_of_df[3] + list_of_df[4])/5

In [82]:
print(test_pred2.shape)

(3492890, 14)


In [83]:
#test_pred2 = pd.DataFrame(np.random.rand(10,14))

In [84]:
test_pred2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.000128,0.001025,0.000093,0.800884,0.064238,0.000251,0.069825,0.000023,0.000095,0.002504,0.000186,0.059474,0.000181,0.001093
1,0.000090,0.006856,0.000096,0.139191,0.033010,0.000199,0.038092,0.000132,0.000521,0.025884,0.001590,0.742009,0.000190,0.012140
2,0.000162,0.013328,0.000180,0.160638,0.089681,0.000364,0.034865,0.006056,0.000513,0.085234,0.004825,0.572058,0.000225,0.031872
3,0.000185,0.002072,0.000386,0.047967,0.022400,0.000375,0.085687,0.005963,0.000379,0.630752,0.000904,0.156562,0.000259,0.046109
4,0.000055,0.001075,0.000061,0.047070,0.232305,0.000116,0.009115,0.000019,0.000085,0.005092,0.000079,0.704745,0.000075,0.000107


In [85]:
temp_columns = ['object_id','class_6','class_15','class_16','class_42','class_52','class_53','class_62','class_64','class_65','class_67','class_88','class_90','class_92','class_95','class_99']

In [86]:
test_pred2.columns = temp_columns[1:15]

In [87]:
def getUnknown(data):
    return ((((((data["mymedian"]) + (((data["mymean"]) / 2.0)))/2.0)) + (((((1.0) - (((data["mymax"]) * (((data["mymax"]) * (data["mymax"]))))))) / 2.0)))/2.0)

feats = ['class_6', 'class_15', 'class_16', 'class_42', 'class_52', 'class_53',
         'class_62', 'class_64', 'class_65', 'class_67', 'class_88', 'class_90',
         'class_92', 'class_95']

In [88]:
klm = pd.DataFrame()
klm['mymean'] = test_pred2[feats].mean(axis=1)
klm['mymedian'] = test_pred2[feats].median(axis=1)
klm['mymax'] = test_pred2[feats].max(axis=1)

In [89]:
test_pred2['class_99'] = getUnknown(klm)

In [90]:
test_pred2.tail()

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
492885,0.000216,0.389886,0.000431,0.198374,0.095011,0.000491,0.150152,0.000264,0.001804,0.017425,0.002311,0.142654,0.000314,0.000668,0.244626
492886,0.000230,0.010590,0.000177,0.043540,0.147001,0.000280,0.019180,0.458607,0.000438,0.037366,0.000803,0.274069,0.000249,0.007470,0.237072
492887,0.000796,0.024077,0.000157,0.894451,0.035766,0.000377,0.030973,0.000047,0.000174,0.001290,0.000448,0.008893,0.000237,0.002314,0.080289
492888,0.000070,0.694377,0.000088,0.165717,0.003054,0.000125,0.010740,0.096651,0.000484,0.000305,0.000194,0.027543,0.000111,0.000542,0.175357
492889,0.000245,0.098533,0.000358,0.447962,0.183607,0.000595,0.065431,0.000096,0.000619,0.025491,0.001374,0.173983,0.000793,0.000911,0.236741


In [91]:
test_pred2 = test_pred2.reset_index(drop=True)

In [92]:
print(test_pred2.shape,test_id.shape)

(3492890, 15) (3492890,)


In [93]:
test_id.tail()

3492885    130787966
3492886    130787971
3492887    130787974
3492888    130788053
3492889    130788054
Name: object_id, dtype: int64

In [94]:
test_id = test_id.reset_index(drop=True)

In [95]:
test_id.index == test_pred2.index

array([ True,  True,  True, ...,  True,  True,  True])

In [96]:
%%time
test_pred = pd.concat([test_id,test_pred2],axis=1)

CPU times: user 76.8 ms, sys: 151 ms, total: 228 ms
Wall time: 226 ms


In [97]:
test_pred = test_pred[temp_columns]

In [98]:
test_pred.head()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
0,13,0.000128,0.001025,0.000093,0.800884,0.064238,0.000251,0.069825,0.000023,0.000095,0.002504,0.000186,0.059474,0.000181,0.001093,0.130663
1,14,0.000090,0.006856,0.000096,0.139191,0.033010,0.000199,0.038092,0.000132,0.000521,0.025884,0.001590,0.742009,0.000190,0.012140,0.157851
2,17,0.000162,0.013328,0.000180,0.160638,0.089681,0.000364,0.034865,0.006056,0.000513,0.085234,0.004825,0.572058,0.000225,0.031872,0.214550
3,23,0.000185,0.002072,0.000386,0.047967,0.022400,0.000375,0.085687,0.005963,0.000379,0.630752,0.000904,0.156562,0.000259,0.046109,0.197197
4,34,0.000055,0.001075,0.000061,0.047070,0.232305,0.000116,0.009115,0.000019,0.000085,0.005092,0.000079,0.704745,0.000075,0.000107,0.171451


In [99]:
print(test_pred.shape)

(3492890, 16)


In [100]:
%%time
test_pred.to_csv('test_pred_40.csv',index=False)

CPU times: user 1min 28s, sys: 971 ms, total: 1min 29s
Wall time: 1min 30s


In [ ]:
#!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_27.csv -m "Message"